<a href="https://colab.research.google.com/github/Daku-on/signate_cup_2024/blob/main/src/00_autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.metrics import roc_auc_score #精度評価に使用

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 各種定数の定義
# 乱数シード
RANDOM_SEED = 42
# target となる特徴量
TARGET_COULMN_NAME = "ProdTaken"
# 削除する特徴量のリスト
DROP_COLUMNS = []
# one-hot encoding する特徴量のリスト
ONE_HOT_ENCODING_COLUMNS = []
# ベストモデルのパス
BEST_MODEL_PATH = "../working/best_autogluon_model.pkl"

In [4]:
# 評価関数の定義
ag_roc_auc_scorer = make_scorer(
    name="roc_auc_score",
    score_func=roc_auc_score,
    optimum=1,
    greater_is_better=True,
    needs_threshold=True,
)

In [14]:
# 学習データの読み込み
# train_df =  pd.read_csv("data/train.csv")
train_df = pd.read_csv("/content/drive/mydrive/signate_cup_2024_data/train.csv")
X_train_df = train_df.drop(columns=[TARGET_COULMN_NAME], inplace=False)

In [15]:
train_df.columns

Index(['id', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'MonthlyIncome',
       'customer_info', 'ProdTaken'],
      dtype='object')

In [7]:
# テストデータの読み込み
# test_df =  pd.read_csv("data/test.csv")
test_df = pd.read_csv("/content/drive/MyDrive/signate_cup_2024_data/test.csv")
X_test_df = test_df

In [19]:
predictor = TabularPredictor(
        label=TARGET_COULMN_NAME,
        eval_metric=ag_roc_auc_scorer,
        path="/content/drive/MyDrive/signate_cup_2024_data/models",
    ).fit(
        train_df,
        # TODO: how to specify the random seed?
        # random_state=RANDOM_SEED,
        # hyperparameters='default',
        # hyperparameter_tune_kwargs={"search_strategy": "bayesopt"},
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.73 GB / 12.67 GB (84.7%)
Disk Space Avail:   67.88 GB / 100.00 GB (67.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ..

In [20]:
y_pred = predictor.predict_proba(X_test_df)

In [21]:
y_pred.head()

,0,1
0,0.857806,0.142194
1,0.839074,0.160926
2,0.701389,0.298611
3,0.661366,0.338634
4,0.763269,0.236731


In [22]:
submission_df = test_df["id"].to_frame()
submission_df["ProdTaken"] = y_pred[1]

In [23]:
submission_df.head()

,id,ProdTaken
0,3489,0.142194
1,3490,0.160926
2,3491,0.298611
3,3492,0.338634
4,3493,0.236731


In [26]:
submission_df.to_csv("submission.csv", index=False, header=None)